In [1]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
TRAINING_DIR = "C:/Users\hp\Downloads/FIRE-SMOKE-DATASET/FIRE-SMOKE-DATASET/Train"
training_datagen = ImageDataGenerator(rescale=1./255,
zoom_range=0.15,
horizontal_flip=True,
fill_mode='nearest')
VALIDATION_DIR = "C:/Users\hp\Downloads/FIRE-SMOKE-DATASET/FIRE-SMOKE-DATASET/Test"
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = training_datagen.flow_from_directory(
TRAINING_DIR,
target_size=(224,224),
shuffle = True,
class_mode='categorical',
batch_size = 128)
validation_generator = validation_datagen.flow_from_directory(
VALIDATION_DIR,
target_size=(224,224),
class_mode='categorical',
shuffle = True,
batch_size= 14)

Found 1800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
input_tensor = Input(shape=(224, 224, 3))
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
  layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(
train_generator,
steps_per_epoch = 14,
epochs = 20,
validation_data = validation_generator,
validation_steps = 14)

Epoch 1/20
14/14 [==============================] - 53s 4s/step - loss: 7.8758 - acc: 0.8020 - val_loss: 0.1225 - val_acc: 0.9439
Epoch 2/20
14/14 [==============================] - 59s 4s/step - loss: 0.0823 - acc: 0.9683 - val_loss: 0.2242 - val_acc: 0.9286
Epoch 3/20
14/14 [==============================] - 54s 4s/step - loss: 0.3626 - acc: 0.8959 - val_loss: 0.1375 - val_acc: 0.9592
Epoch 4/20
14/14 [==============================] - 53s 4s/step - loss: 0.2689 - acc: 0.9103 - val_loss: 0.1011 - val_acc: 0.9643
Epoch 5/20
14/14 [==============================] - 53s 4s/step - loss: 0.1757 - acc: 0.9384 - val_loss: 0.1329 - val_acc: 0.9592
Epoch 6/20
14/14 [==============================] - 53s 4s/step - loss: 0.1494 - acc: 0.9569 - val_loss: 1.1256 - val_acc: 0.6684
Epoch 7/20
14/14 [==============================] - 53s 4s/step - loss: 0.1272 - acc: 0.9575 - val_loss: 0.1818 - val_acc: 0.9337
Epoch 8/20
14/14 [==============================] - 54s 4s/step - loss: 0.1761 - acc: 0.94

In [3]:
for layer in model.layers[:249]:
  layer.trainable = False
for layer in model.layers[249:]:
  layer.trainable = True
#Recompile the model for these modifications to take effect
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(
train_generator,
steps_per_epoch = 14,
epochs = 10,
validation_data = validation_generator,
validation_steps = 14)

Epoch 1/10
14/14 [==============================] - 84s 6s/step - loss: 0.2910 - acc: 0.8780 - val_loss: 0.1586 - val_acc: 0.9643
Epoch 2/10
14/14 [==============================] - 83s 6s/step - loss: 0.2217 - acc: 0.9103 - val_loss: 0.1390 - val_acc: 0.9592
Epoch 3/10
14/14 [==============================] - 83s 6s/step - loss: 0.1515 - acc: 0.9420 - val_loss: 0.1306 - val_acc: 0.9643
Epoch 4/10
14/14 [==============================] - 84s 6s/step - loss: 0.1115 - acc: 0.9581 - val_loss: 0.1256 - val_acc: 0.9694
Epoch 5/10
14/14 [==============================] - 84s 6s/step - loss: 0.0882 - acc: 0.9677 - val_loss: 0.1225 - val_acc: 0.9694
Epoch 6/10
14/14 [==============================] - 84s 6s/step - loss: 0.0836 - acc: 0.9677 - val_loss: 0.1209 - val_acc: 0.9694
Epoch 7/10
14/14 [==============================] - 83s 6s/step - loss: 0.0704 - acc: 0.9791 - val_loss: 0.1198 - val_acc: 0.9643
Epoch 8/10
14/14 [==============================] - 82s 6s/step - loss: 0.0618 - acc: 0.98

In [4]:
model.save("model.h5")

In [7]:
!pip install opencv-python

In [13]:
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from keras.preprocessing import image
model = tf.keras.models.load_model('model.h5')
video = cv2.VideoCapture(0)
while True:
        _, frame = video.read()
        im = Image.fromarray(frame, 'RGB')
        im = im.resize((224,224))
        img_array = image.img_to_array(im)
        img_array = np.expand_dims(img_array, axis=0) / 255
        probabilities = model.predict(img_array)[0]
        prediction = np.argmax(probabilities)
        if prediction == 0:
                #frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
                #print(probabilities[prediction])
                print(1)
        cv2.imshow("Capturing", frame)
        
        # provide IoT quit identifier
        key=cv2.waitKey(1)
        if key == ord('q'):
                break
video.release()
cv2.destroyAllWindows()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
